In [1]:
import numpy as np
import itertools

In [11]:
#constantes
Kb = 8.6173324e-5

In [4]:
def possible_matrices(size, N_a):
    matrices = []
    # Define the counts of A and B
    N_b = (size * size) - N_a

    # Generate all possible arrangements of "A" and "B" in the matrix
    elements = ['A'] * N_a + ['B'] * N_b
    arrangements = set(itertools.permutations(elements, size * size))

    # Iterate over each arrangement and reshape it into a matrix
    for arrangement in arrangements:
        matrix = np.array(arrangement).reshape(size, size)
        matrices.append(matrix)
    
    return matrices

In [6]:
matrix = possible_matrices(3,3)[0]
print(matrix)

[['B' 'B' 'A']
 ['B' 'B' 'B']
 ['A' 'A' 'B']]


In [9]:
def determiar_vizinhos(matriz, size):

    # Array of offsets for neighboring elements
    offsets = np.array([[0, 1], [1, 0], [0, -1], [-1, 0]])

    # Generate indices for all elements in the matrix
    i, j = np.indices((size, size))

    # Reshape offsets to have an extra dimension for broadcasting
    reshaped_offsets = offsets.reshape(1, 1, -1, 2)

    # Calculate indices of neighboring elements
    neighbor_indices = (i[..., np.newaxis, np.newaxis] + reshaped_offsets) % np.array([size,size])

    # Create a dictionary to store neighbors
    vizinhos = {(x, y): matriz[neighbor_indices[x, y, :, 0], neighbor_indices[x, y, :, 1]].tolist() for x in range(size) for y in range(size)}

    return vizinhos


In [10]:
determiar_vizinho = determiar_vizinhos(matrix,3)
print(determiar_vizinho)

{(0, 0): ['B', 'B', 'A', 'A'], (0, 1): ['B', 'B', 'A', 'A'], (0, 2): ['B', 'B', 'A', 'A'], (1, 0): ['B', 'A', 'B', 'B'], (1, 1): ['B', 'A', 'B', 'B'], (1, 2): ['B', 'A', 'B', 'B'], (2, 0): ['A', 'A', 'A', 'B'], (2, 1): ['A', 'A', 'A', 'B'], (2, 2): ['A', 'A', 'A', 'B']}


In [12]:
def helmholtz(energias, temperatura):
    """Essa função calcula o valor da função de partição
    
    Args:
        
        energias: lista com a energia de cada configuração da célula.
        temperatura: temperatura na qual estaria a célula.
        
    Return:
    
        Z: valor da função de partição em uma temperatura.
    """
    beta = 1.0 / (Kb * temperatura)
    Z = 0.0
    for energia in energias:
        Z += np.exp(-beta * energia)

    F = beta * np.log(Z)
    
    return F